# This code is taken from the ginicoeff file in the repo

In [13]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [14]:
import requests
host = 'https://api.census.gov/data'
year = '/2015'
dataset_acronym = '/acs/acs1'
g = '?get='
variables = 'NAME,B19083_001E'
location = '&for=state:*&key='
usr_key = "0d20bc54b2f22dccdd48690f3ef9c523a9b7da70"
query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
response = requests.get(query_url)
contents = response.text

In [15]:
contents

'[["NAME","B19083_001E","state"],\n["Mississippi","0.4759","28"],\n["Missouri","0.4634","29"],\n["Montana","0.4623","30"],\n["Nebraska","0.4473","31"],\n["Nevada","0.4548","32"],\n["New Hampshire","0.4347","33"],\n["New Jersey","0.4832","34"],\n["New Mexico","0.4795","35"],\n["New York","0.5138","36"],\n["North Carolina","0.4776","37"],\n["North Dakota","0.4664","38"],\n["Ohio","0.4637","39"],\n["Oklahoma","0.4702","40"],\n["Oregon","0.4622","41"],\n["Pennsylvania","0.4685","42"],\n["Rhode Island","0.4732","44"],\n["South Carolina","0.4698","45"],\n["South Dakota","0.4436","46"],\n["Tennessee","0.4789","47"],\n["Texas","0.4823","48"],\n["Vermont","0.4452","50"],\n["Utah","0.4252","49"],\n["Virginia","0.4681","51"],\n["Washington","0.4561","53"],\n["West Virginia","0.4581","54"],\n["Wisconsin","0.4410","55"],\n["Wyoming","0.4374","56"],\n["Puerto Rico","0.5589","72"],\n["Alabama","0.4811","01"],\n["Alaska","0.4322","02"],\n["Arizona","0.4702","04"],\n["Arkansas","0.4773","05"],\n["Calif

In [16]:
gini = pd.read_json(contents)

In [17]:
gini

,0,1,2
0,NAME,B19083_001E,state
1,Mississippi,0.4759,28
2,Missouri,0.4634,29
3,Montana,0.4623,30
4,Nebraska,0.4473,31
5,Nevada,0.4548,32
6,New Hampshire,0.4347,33
7,New Jersey,0.4832,34
8,New Mexico,0.4795,35
9,New York,0.5138,36


In [24]:
def create_gini():
    years = ["/2011", "/2012", "/2013", "/2014", "/2015", "/2016", "/2017", "/2018", "/2019", "/2021", 
             "/2022"]
    year = "/2010"
    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    response = requests.get(query_url)
    contents = response.text
    gini = pd.read_json(contents)
    year_col = np.full(len(gini), year[1:])
    year_col = pd.DataFrame(year_col)
    gini = pd.concat([gini, year_col], axis = 1)
    gini.columns = ["State","gini","state", "year"]
    gini = gini[gini["State"] != "Puerto Rico"]
    gini = gini[gini["State"] != "District of Columbia"]
    for year in years:
        query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
        response = requests.get(query_url)
        contents = response.text
        temp = pd.read_json(contents)
        year_col = np.full(len(temp), year[1:])
        year_col = pd.DataFrame(year_col)
        temp = pd.concat([temp, year_col], axis = 1)
        temp.columns = ["State","gini","state", "year"]
        temp = temp.sort_values("State")
        temp = temp[temp["State"] != "District of Columbia"]
        temp = temp[temp["State"] != "Puerto Rico"]
        gini = pd.concat([gini, temp])
    return gini

In [25]:
gini = create_gini()

In [26]:
gini

,State,gini,state,year
0,NAME,B19083_001E,state,2010
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [27]:
gini = gini[gini["State"] != "NAME"]
gini.columns = ["State","gini","state", "year"]
gini = gini[gini["State"] != "NAME"]
gini = gini.drop(["state"], axis = 1)

In [28]:
coeffs = pd.to_numeric(gini["gini"])
gini["gini"] = coeffs
years = pd.to_numeric(gini["year"])
gini["year"] = years

In [29]:
gini

,State,gini,year
1,Alabama,0.4720,2010
2,Alaska,0.4220,2010
3,Arizona,0.4550,2010
4,Arkansas,0.4580,2010
5,California,0.4710,2010
...,...,...,...
47,Virginia,0.4755,2022
48,Washington,0.4742,2022
49,West Virginia,0.4804,2022
50,Wisconsin,0.4510,2022


# Folium maps (Pop, Gini, Min Wage, % w/ College Degree)

In [30]:
import folium
import pandas as pd

statespopulation = {"State": ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                              "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                              "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
                              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
                              "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
                              "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
    "Value": [4785298.0, 713985.0,6413737.0,2921606.0,37349363.0,
              5049071.0,3577073.0,899769.0,18843326.0,9712587.0,
              1363621.0,1571450.0,12843166.0,6490621.0,3049883.0,
              2859169.0,4346266.0,4544228.0,1327567.0,5785982.0,
              6557254.0,9877574.0,5310584.0,2970036.0,5996231.0,
              990898.0,1830429.0,2704642.0,1316759.0,8801624.0,
              2065932.0,19392283.0,9561558.0,674499.0,
              11536182.0,3761702.0,3838957.0,12709630.0,
              1052886.0,4636312.0,816463.0,6356897.0,25257114.0,
              2776469.0,625960.0,8024617.0,6744496.0,1853973.0,
              5691047.0,564460.0]}

statespopulation = pd.DataFrame(statespopulation)
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(geo_data="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json",
                  name="c/horopleth",
                  data=statespopulation,
                  columns=["State", "Value"],
                  key_on="feature.properties.name",
                  fill_color="BuPu",fill_opacity=1.0,
                  line_opacity=0.6,
                  legend_name="Population Per State",).add_to(m)

folium.LayerControl().add_to(m)
m

In [31]:
stateginicoe = {"State": ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                              "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                              "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
                              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
                              "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
                              "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
    "Value": [0.472,0.422,0.455,0.458,0.471,0.457,
              0.486,0.44,0.474,0.468,0.433,0.433,0.465,0.44,
              0.427,0.445,0.466,0.475,0.437,0.443,0.475,0.451,
              0.44,0.468,0.455,0.435,0.432,0.448,0.425,0.464,
              0.464,0.499,0.464,0.433,0.452,0.454,0.449,
              0.461,0.467,0.461,0.442,0.468,0.469,0.419,0.444,
              0.459,0.441,0.451,0.43,0.423]}

#here I typed in all the values with accordance to each state. 
#Taken from the 6th cell from gini
stateginicoe = pd.DataFrame(stateginicoe)
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(geo_data="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json",
                  name="c/horopleth",
                  data=stateginicoe,
                  columns=["State", "Value"],
                  key_on="feature.properties.name",
                  fill_color="BuPu",fill_opacity=1.0,
                  line_opacity=0.6,
                  legend_name="Gini Coefficient per state",).add_to(m)

folium.LayerControl().add_to(m)
m

**Folium map of Gini Coefficient Per State**

Gini coeffient shows wealth inequality scaled from 0 to 1. 1 being higher wealth inequality. You can see a pattern of possible trends within the south having higher wealth inequality distributions opposed to the north. Not significant but states with greater population tend to produced a larger wealth gap.

In [42]:
statespopcollege = {"State": ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                              "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                              "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
                              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
                              "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
                              "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
                   "Value": [9.191966728090920, 11.673354482237000, 10.787611029264200, 8.655787262211260,
                             12.345257401043200, 15.42967013139650, 13.753982655651700, 11.047279912955400,
                             11.525969459956300, 11.260048429939400, 13.520178994016700, 10.513729358236000,
                             12.688654806766500, 9.508997675260960, 11.310335511231100, 12.39447545772920,
                             8.281361518139940, 9.332146186326920, 12.229137964411600, 13.185609633766600,
                             15.132965720101700, 10.36448828426900, 14.311288551315600, 7.978522819252020,
                             10.66256453428830, 13.45809558602400, 12.692434396526700, 9.486985708274890,
                             14.059444438959600, 15.01923963123170, 9.196285260115050, 12.547934660400700,
                             11.746181950682100, 12.959841304434800, 10.493307057742300, 10.01142567912080,
                             12.45476310362420, 11.371086333748500, 12.174917322483200, 10.446686935650600,
                             12.096077838187400, 9.770914960553870, 10.823568361769300, 11.367928113009700,
                             14.000575116620900, 13.406072838118000, 13.312840574002900, 7.597198017446860,
                             11.594562476816700, 10.316408602912500]}

statespopcollege = pd.DataFrame(statespopcollege)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json",
    name="choropleth",
    data=statespopcollege,
    columns=["State", "Value"],
    key_on="feature.properties.name",
    fill_color="OrRd",
    fill_opacity=1.0,
    line_opacity=0.6,
    legend_name="% of the state population with a college degree",
).add_to(m)

folium.LayerControl().add_to(m)
m

In [43]:
stateminwage = {"State": ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                              "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                              "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
                              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
                              "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
                              "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
    "Value": [7.25,10.85,13.85,11.00,15.50,13.65,15.00, 11.75,
             12.00,7.25,12.00,7.25,13.00,7.25,7.25,7.25,7.25,
             7.25,13.80,13.25,15.00,10.10,10.59,7.25,
             12.00,9.95,10.50,10.25,7.25, 14.13,12.00,
             14.20,7.25,7.25,10.10,7.25,13.50,7.25,
             13.00,7.25,10.80,7.25,7.25,7.25,13.18,
             12.00,15.74,8.75,7.25,7.25]}
stateminwage = pd.DataFrame(stateminwage)
m = folium.Map(location=[48, -102], zoom_start=2.5)
folium.Choropleth(geo_data="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json",
                  name="choropleth",
                  data=stateminwage,
                  columns=["State", "Value"],
                  key_on="feature.properties.name",
                  fill_color="BuGn",fill_opacity=1.0,
                  line_opacity=0.6,
                  legend_name="Minimum wage (in USD $)",).add_to(m)

folium.LayerControl().add_to(m)
m

#citation: https://www.hrministrysolutions.com/blog/2023-minimum-wage?gad_source=1&gclid=EAIaIQobChMIibKN-YT0ggMV_mJHAR3fdAapEAAYASAAEgJ7fvD_BwE
# accessed dec 2nd, 2023
# I wanted to represent the most current 2023 values opposed to the averages of 2010-2022 from the previous datasets

In [38]:
stateperc_unemp = {"State": ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
                              "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                              "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi",
                              "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
                              "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
                              "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"],
    "Value": [10.375,8.133333333333330,10.3,7.858333333333330,12.458333333333300,
              9.141666666666670,9.558333333333330,8.658333333333330,10.825,
              10.658333333333300,6.825,8.758333333333330,10.466666666666700,
              10.083333333333300,6.116666666666670,6.933333333333330,
              10.266666666666700,7.516666666666670,8.416666666666670,
              7.825,8.058333333333330,12.191666666666700,7.391666666666670,
              10.083333333333300,9.508333333333330,7.183333333333330,4.7,13.733333333333300,6.083333333333330,
              9.658333333333330,7.775000000000000,8.716666666666670,10.716666666666700,
              3.6916666666666700,10.266666666666700,6.5,10.691666666666700,
              8.166666666666670,11.625,11.3,4.908333333333330,9.825,8.191666666666670,
              7.933333333333330,6.316666666666670,7.291666666666670,9.133333333333330,
              8.633333333333330,8.5,6.941666666666670]}

stateperc_unemp = pd.DataFrame(stateperc_unemp)
m = folium.Map(location=[48, -102], zoom_start=3)
folium.Choropleth(geo_data="https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json",
                  name="choropleth",
                  data=stateperc_unemp,
                  columns=["State", "Value"],
                  key_on="feature.properties.name",
                  fill_color="PuBuGn",fill_opacity=1.0,
                  line_opacity=0.6,
                  legend_name="Unemployment rate per state",).add_to(m)

folium.LayerControl().add_to(m)
m
